In [96]:
import json
from os import walk
from collections import Counter

In [97]:
class Brick():
    def __init__(self, full_path, name, files):
        self.full_path = full_path
        self.files = files
        self.name = name
        self._setFiles()
        self._setTitle()
        self._setDescription()

    def _setFiles(self):
        for file in files:
            if 'README.md' == file:
                with open(self.full_path +'/'+ file, 'r') as myfile:
                    self.readme = [line.strip() for line in myfile.readlines()]
            if file.endswith('.py'):
                with open(self.full_path +'/'+ file, 'r') as myfile:
                    self.script = myfile.read()
            if 'LICENSE.md' == file:
                with open(self.full_path +'/'+ file, 'r') as myfile:
                    lines = myfile.readlines()
                    self.copyright, self._show_attribution = self._getLicense(lines)

    def _setTitle(self):
        first_line = self.readme[0]
        i = first_line.find('## ')
        self.title = first_line[3:] if i != -1 else 'error: somethings wrong'

    def _setDescription(self):
        self.description = '\n'.join(self.readme[5:])

    def _firstcell(self):
        markdown_cell = {'cell_type': 'markdown',
            'metadata': {},
            'source': ['---\n',
            'title: "'+self.title+'"\n',
            'description: "'+self.description+'"\n',
            'tags: \n',
            'URL: https://github.com/ActiveState/code\n',
            'Licence: '+self.copyright+'\n',
            'Creator: \n',
            'Meta: \n',
            '\n',
            '---']}

        return markdown_cell

    def write_to_file(self):
        with open ('./template.ipynb') as json_file:
            data = json.load(json_file)

            data['cells'].insert(0, self._firstcell())
            data['cells'][2]['source'] = self.script
            
            if self._show_attribution:
                first = {'cell_type': 'markdown', 'metadata': {}, 'source': ['# Attribution&']}
                second = {'cell_type': 'markdown', 'metadata': {}, 'source': [self.copyright]}
                data['cells'].insert(3, first)
                data['cells'].insert(4, second)
                

        with open ('../inprogress/'+str(self.name)+'.ipynb', 'w') as outfile:
            json.dump(data, outfile)

    def _getLicense(self, lines):
        if lines[0].startswith('Copyright'):
            hold, attr = self._licenseType(' '.join(lines[2:]).replace('\n',''))
            copyright = lines[0].strip() + '. ' + hold
        else:
            copyright, attr = 'CC0 (Creative Commons) License', False
        return copyright, attr

    def _licenseType(self, line):
        # 2236 
        if 'PSF' in line:
            return 'Python Software Foundation License', False
        # 2094
        if line.startswith('Permission is hereby granted'):
            return '''Permission is hereby granted, free of charge, to any person 
obtaining a copy of this software and associated documentation files (the "Software"), to deal in the 
Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, 
distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is 
furnished to do so, subject to the following conditions:  The above copyright notice and this permission 
notice shall be included in all copies or substantial portions of the Software.  THE SOFTWARE IS PROVIDED 
"AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF 
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR 
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, 
TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS 
IN THE SOFTWARE.''', True
        # 113
        if line.startswith('Redistribution and use'):
            return '''Redistribution and use in source and binary forms, with or without modification,
are permitted provided that the following conditions are met:  1. Redistributions of source code must 
retain the above copyright notice, this list of conditions and the following disclaimer.  2. 
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and 
the following disclaimer in the documentation and/or other materials provided with the distribution. 
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED
WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A
PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR
ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED
TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION)
HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING
NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE 
POSSIBILITY OF SUCH DAMAGE.''', True

        # 84
        if 'GNU Lesser General Public License' in line:
            return 'GNU Lesser General Public License v. 2.1', False
        # 23
        if 'GNU General Public License' in line:
            return 'GNU General Public License v. 2.0', False
        # 11
        if 'Apache' in line:
            return 'Apache License (Version 2.0)', False
        # 1
        if line.startswith('--------'):
            return 'The Artistic License 2.0', False
        # 1
        if 'Mozilla' in line:
            return 'Mozilla Public License, v. 2.0', False
        # 0
        else:
            print(line)
            return 'should never happen', False

In [98]:
mypath='/Users/jimmydelano/active-state/code/recipes/Python/'

bricks = []
for (dirpath, _, files) in walk(mypath):
    if len(dirpath) == len(mypath):
        continue

    name = dirpath[len(mypath)+1:]
    
#     with open(dirpath +'/'+ 'LICENSE.md', 'r') as myfile:
#         lines = myfile.readlines()
#         copyright = getLicense(lines)
        
    b = Brick(dirpath, name, files)
    bricks.append(b)

In [99]:
for brick in bricks:
    brick.write_to_file()

In [100]:
%%bash
cd ~/cocode/scripts/
python3 midas.py